In [1]:
# dependencias
%pip install agentpy IPython

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import agentpy as ap
import IPython
import numpy as np
import heapq
import matplotlib.pyplot as plt

class HarvestModel(ap.Model):
    def a_star(self, grid, start, goal):
      def heuristic(a, b):
          return abs(a[0] - b[0]) + abs(a[1] - b[1])

      directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
      open_list = []
      heapq.heappush(open_list, (0, start))

      g_score = {start: 0}
      f_score = {start: heuristic(start, goal)}
      came_from = {}

      while open_list:
          _, current = heapq.heappop(open_list)
          if current == goal:
              path = []
              while current in came_from:
                  path.append(current)
                  current = came_from[current]
              path.append(start)
              path.reverse()
              return path

          for direction in directions:
              neighbor = (current[0] + direction[0], current[1] + direction[1])

              if 0 <= neighbor[0] < len(grid) and 0 <= neighbor[1] < len(grid[0]) and grid[neighbor[0]][neighbor[1]] != (1 or 5 or 10):
                      tentative_g_score = g_score[current] + 1

                      if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                          came_from[neighbor] = current
                          g_score[neighbor] = tentative_g_score
                          f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                          heapq.heappush(open_list, (f_score[neighbor], neighbor))
      return None

    def setup(self):
        self.truck_agents = [Harvester(self, ((self.p['size'][0] - 2), 1)), Collecter(self, ((self.p['size'][0] - 2), 0))]
        self.aux_grid_matrix = np.zeros(self.p.size, dtype=int)

        self.path = []  # Store the A* path here
        self.path_index = 0  # Track current position in the A* path
        self.collecting = False  # Track whether the collector is moving along the A* path

        self.custom_t = 0

        self.lista_coordenadas = []

        for i in range(1, (self.p['size'][0] - 1), 1):
          if i % 2 != 0:
            for j in range((self.p['size'][0] - 2), 0, -1):
              self.lista_coordenadas.append((j, i))
          else:
            for j in range(1, (self.p['size'][0] - 1), 1):
              self.lista_coordenadas.append((j, i))

        self.truck_agents[1].capacity = self.p['capacity'] # capacidad inicial
        # Mark the grid with where the harvester should pass
        for i in range(len(self.lista_coordenadas)):
            self.aux_grid_matrix[self.lista_coordenadas[i]] = 1

    def step(self):
      if self.collecting:
          # Move collector along A* path one step at a time
          if self.path_index < len(self.path):
              step = self.path[self.path_index]

              self.truck_agents[1].last_index = self.truck_agents[1].index
              self.truck_agents[1].index = step

              # Update the grid
              # Clear the last position
              if self.truck_agents[1].last_index != self.truck_agents[1].index:
                  self.aux_grid_matrix[self.truck_agents[1].last_index] = 0

              # Set the new position
              self.aux_grid_matrix[self.truck_agents[1].index] = 10

              self.path_index += 1  # Move to the next step in the path

          if self.path_index == len(self.path):
              # Finished moving along the A* path
              self.truck_agents[1].capacity = self.p['capacity']

              self.collecting = False

      else:
          if self.truck_agents[1].capacity == 0 and self.custom_t > 0:
              start = self.truck_agents[1].index
              goal = (0, self.p['size'][0] - 1)
              path_to_goal = self.a_star(self.aux_grid_matrix, start, goal) or []
              path_back = self.a_star(self.aux_grid_matrix, goal, start) or []

              # Combine the path to the goal and back to the start
              self.path = path_to_goal + path_back

              # Remove the first coordinate from the path (usually the starting point)
              if len(self.path) > 1:
                  self.path = self.path[1:]

              self.path_index = 0  # Reset path index
              self.collecting = True  # Start moving the collector along the path
          else:


              path_to_tractor = self.a_star(self.aux_grid_matrix, self.truck_agents[1].index, self.truck_agents[0].index)
                         
              # MATRIX 
              # 1 = crops
              # 5 = tractor
              # 10 = collector
              # 0 = empty space

              # Normal movement of harvester and collector
              self.custom_t += 1
              self.truck_agents[0].last_index = self.truck_agents[0].index
              self.truck_agents[1].last_index = self.truck_agents[1].index

              self.truck_agents[0].index = self.lista_coordenadas[self.custom_t]
              
              if path_to_tractor:
                if path_to_tractor[-2] != self.truck_agents[0].index:
                  self.truck_agents[1].index = path_to_tractor[-2]

              # Clear the last positions
              self.aux_grid_matrix[self.truck_agents[0].last_index] = 0
              self.aux_grid_matrix[self.truck_agents[1].last_index] = 0

              # Set the new positions
              self.aux_grid_matrix[self.truck_agents[0].index] = 5
              self.aux_grid_matrix[self.truck_agents[1].index] = 10

              self.truck_agents[1].capacity -= 1


      #print(self.aux_grid_matrix)

    # el modelo termina cuando el tractor llega a su posición final
    def update(self):
        if (self.truck_agents[0].index == ((self.p['size'][0] - 2), (self.p['size'][0] - 2))):
          self.stop()

class Harvester(ap.Agent):
    def setup(self, index):
        # posición del harvester en formato de tupla
        self.index = index # posición actual
        self.last_index = index # posición del paso anterior, se usa para marcar con 0 el paso anterior

class Collecter(ap.Agent):
    def setup(self, index):
        # del colelcter en formate de tupla
        self.index = index # posición actual
        self.last_index = index # posición del paso anterior, se usa para marcar con 0 el paso anterior

        self.capacity = 0 # capacidad del colector, ira reduciendo -1 cada vez que camina junto al tractor

parameters = {
    'steps': 100,
    #'truck_track_matrix': lista_coordenadas,
    'size': (6, 6), # tamaño de la matriz
    'capacity': 5 # capacidad del colector
}

def my_plot(model, ax):
    ax.set_title(f'Step: {model.t}')
    ax.grid(True)
    ax.clear()

    ax.imshow(model.aux_grid_matrix, cmap='ocean', interpolation='nearest')

# Initialize the model and figure
fig, ax = plt.subplots()
model = HarvestModel(parameters)
# results = model.run()

# Run the animation
animation = ap.animate(model, fig, ax, my_plot)
IPython.display.HTML(animation.to_jshtml(fps=3))